## QnA with Pubmed using ThirdAI's Playground

In this notebook, you will be able to

1. Download ThirdAI's BOLT LLM trained on Pubmed-800K and the processed data

2. Ask any question and get relevant references from Pubmed.

3. (Optional) How to use your OpenAI key to generate grounded answers without hallucination.

In [ ]:
!pip3 install thirdai==0.7.6
!pip3 install numpy
!pip3 install pandas
!pip3 install openai
!pip3 install paper-qa
!pip3 install langchain
!pip3 install transformers

In [ ]:
from thirdai import bolt,licensing

## Please request for a trial license @ https://www.thirdai.com/try-bolt/
your_thirdai_key = "" 
try:
    licensing.activate(your_thirdai_key)
except:
    pass

import pandas as pd
import numpy as np
import os
import json

In [29]:
#### Model Checkpoint
checkpoint = "pubmed_800k.bolt"
if not os.path.exists(checkpoint):
    os.system("wget -O pubmed_800k.bolt https://www.dropbox.com/s/kwoqt5c7bqbisbl/pubmed_800k.bolt?dl=0")

model = bolt.UniversalDeepTransformer.load(checkpoint)

### Processed Data to show references
display_data = 'pubmed_800k_combined.json'
if not os.path.exists(display_data):
    os.system("wget -O pubmed_800k_combined.json https://www.dropbox.com/s/8phkx4fht9j2npy/pubmed_800k_combined.json?dl=0")

data_store = {}
with open(display_data, "r") as f:
    data = json.load(f)

for json_data in data:
    data_store[json_data["label"]] = json_data

### Get Answers from OpenAI

In this section, we will show how to use LangChain and query OpenAI's QnA module to generate an answer from the references that you retrieve from the above BOLT model. You'll have to specify your own OpenAI key for this module to work. You can replace this segment with any other generative model of your choice. You can choose to use an on-prem open source model like MPT or Dolly for answer generation with the same prompt that you use with OpenAI.

In [11]:
from langchain.chat_models import ChatOpenAI
from paperqa.qaprompts import qa_prompt, make_chain

# Please specify your openai key
your_openai_key = ""

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo', 
    temperature=0.1, 
    openai_api_key=your_openai_key,
)

qa_chain = make_chain(prompt=qa_prompt, llm=llm)

In [33]:
MAX_ARTICLES = 3

## This tokenizer is just used to tokenize the data and pass it to BOLT model
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def get_references(query):
    scores = model.predict({"QUERY": " ".join(map(str, tokenizer.encode(query.lower())))})
    sorted_article_ids = scores.argsort()[-MAX_ARTICLES:][::-1]
    references = [data_store[article_id]['abstract'] for article_id in sorted_article_ids]
    return references

def get_answer(query, references):
    return qa_chain.run(question=query, context_str='\n\n'.join(references), length="abt 50 words")

### Example Question 1

In [34]:
query = "what percentage of cancer patients have depression?"

references = get_references(query)
print(references)

['Background \n Depression is the most common psychiatric comorbidity among people living with HIV/AIDS (PLWHA). Little is known about the comparative effectiveness between different types of antidepressants used to treat depression in this population. We compared the effectiveness of dual-action and single-action antidepressants in PLWHA for achieving remission from depression. \n \n \n Methods \n We used data from the Centers for AIDS Research Network of Integrated Clinic Systems to identify 1,175 new user dual-action or single-action antidepressant treatment episodes occurring from 2005–2014 for PLWHA diagnosed with depression. The primary outcome was remission from depression defined as a Patient Health Questionnaire-9 (PHQ-9) score <5. Mean difference in PHQ-9 depressive symptom severity was a secondary outcome. The main approach was an intent-to-treat (ITT) evaluation complemented with a per protocol (PP) sensitivity analysis. Generalized linear models were fitted to estimate tre

In [35]:
answer = get_answer(query, references)

print(answer)

Approximately 47% of patients with cancer of the breast, prostate, colon/rectum, or lung were defined as having depressive symptoms (Passik et al., 2010).


### Example Question 2

In [14]:
query = "How to detect depression in geriatric cancer patients ?"

references = get_references(query)
answer = get_answer(query, references)
print(answer)

Three popular self-report measures of depression, the Geriatric Depression Scale-Short Form, the Hospital Anxiety and Depression Scale, and the Center for Epidemiological Studies Depression Scale—Revised, were examined for their ability to assess depression in a geriatric cancer setting. These measures produced good internal consistency and were better than chance at predicting depression in a geriatric cancer sample. However, the published cutoff scores for detecting major depression produced inadequate sensitivity, suggesting these scores will miss as many as 33%–83% of geriatric cancer patients who are depressed. Revised cutoff scores were lower than published cutoff scores. The Center for Epidemiological Studies Depression Scale—Revised appeared to have the most utility (Foley2016).
